In [1]:
%cd ~/github/agn_lf/
import source.astro_functions as af
import source.table_functions as tf
import source.lf_xi2 as xi2
import source.vmax as vmax
import source.utils as utils
import source.selection_criteria as sc
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.table as table
import astropy.units as u

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/runburg/github/agn_lf


In [2]:
xmmlss = fits.open('./data/XMM-LSS_20190328.fits')

In [3]:
xmmlss[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                 3394 / width of table in bytes                        
NAXIS2  =              8705837 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                  475 / number of columns                              
EXTNAME = './data/tiles/xmm-lss_20190328_0.fits' / table name                   
TTYPE1  = 'help_id '           / label for column 1                             
TFORM1  = '27A     '           / format for column 1                            
TTYPE2  = 'field   '           / label for column 2                             
TFORM2  = '18A     '        

In [ ]:
# catalog = table.Table.read('./data/SWIRE_and_XSERVS.fits')
file = './data/master_catalogue_xmm-lss_20180221_photoz_20180518_r_optimised.fits'
catalog = table.Table.read(file)
print(len(catalog))
catalog[0]
catalog = table.Table.read('./data/XMM-LSS_20190328.fits')

6152920


In [ ]:
flux36 = 'F_CH1'
flux45 = 'F_CH2'
flux58 = 'flux_ap2_58'
flux80 = 'flux_ap2_80'
# print(catalog[flux58][catalog[flux58] < 0])
never_nan = (np.nan_to_num(catalog[flux58], nan=-99) > 0) & (np.nan_to_num(catalog[flux80], nan=-99) > 0)
fig, axs = sc.plot_wedge(catalog[never_nan], flux36, flux45, flux58, flux80, selection=['lacy05', 'stern05', 'donley12'])
selected_agn = sc.select_ir(catalog[never_nan], flux36, flux45, flux58, flux80, selection_cuts='lacy05')

detection80 = np.nan_to_num(catalog[flux80], nan=-99) > 0
plt.scatter(catalog[detection80]['RA_1'], catalog[detection80]['DEC_1'], s=1, alpha=0.1)

In [ ]:
print(np.sum(never_nan), len(selected_agn))
ct_lacy = catalog[never_nan][selected_agn]

In [ ]:
cosmo = af.setup_cosmology()
print(len(ct_lacy))

ct_lacy = ct_lacy[(ct_lacy['z_eazy'] > 0 ) | (ct_lacy['zSpec'] > 0)]
# ct_lacy = ct_lacy[ct_lacy['z_eazy'] > 0]
print(len(ct_lacy))

import astropy.units as u

spectral_index = 1.5

# unit conversion: ujansky -> jansky: 1e-6
# jansky -> erg / s / cm**2 / Hz: 1e-23
# erg / s / Hz -> erg / s: nu = c/lambda = 3e8 / 3.6e-6
unit_conversion = 1e-6 * 1e-23 * 3e8 / 3.6e-6 * (3.086e24)**2
unit_conversion = 3 / 3.6 * 3.086**2 * 1e33
# print(unit_conversion)

z = np.array([row['zSpec'] if row['zSpec'] > 0 else row['z_eazy'] for row in ct_lacy])
# z = ct_lacy['z_eazy']
ldist = cosmo.luminosity_distance(z).value**2
# print(ldist)
l = 4 * np.pi * cosmo.luminosity_distance(z).value**2 * ct_lacy[flux58] * unit_conversion / (1 + z)**(1 + spectral_index)
# print(l)
print(np.log10(l))

In [ ]:
print(min(l), max(l), min(z), max(z))
z_sample_min = 0.5
z_sample_max = 4
l_sample_min = 3e42
l_sample_max = 3e46

good_redshift_and_l_selected = (l_sample_min < l) & (l_sample_max > l) & (z_sample_min < z) & (z_sample_max > z)
l = l[good_redshift_and_l_selected]
z = z[good_redshift_and_l_selected]
print(len(z))

In [ ]:
num_bins_z = 10
num_bins_l = 12
# redshift range 
z_bins = np.logspace(np.log10(z_sample_min)-0.01, np.log10(z_sample_max)+0.01, num=num_bins_z)
print(z_bins)
l_bins = np.logspace(np.log10(l_sample_min), np.log10(l_sample_max), num=num_bins_l)

# print(len(z))
fig, ax = vmax.l_z_histo(l, z, l_bins, z_bins, band='2-10 keV X-ray', unit=r'erg s$^{-1}$')

In [ ]:
image_file = './data/swire_XMM_I3_tile_1_1_v4_cov.fits'
wcs, hdu = utils.load_wcs(image_file)
fig, ax = vmax.exposure_plot(wcs, hdu.data, survey='SWIRE', band='IRAC 3')

In [ ]:
# flux_limit = 5 * unit_conversion
flux_limit = 3631 * 10**(23/-2.5) * 5.8e-6 / 3e8
print(flux_limit)
zmax = vmax.compute_zmax(l, z, cosmo, flux_limit, zspacing=0.5)
zmin = np.array([0]*len(zmax))

full_fluxes = catalog[never_nan][flux58]
# print(len(full_fluxes), len(ct_lacy), len(good_redshift_and_l_selected))
coverage_correction = vmax.coverage_correction(full_fluxes, ct_lacy[flux58][good_redshift_and_l_selected])

# image_file = './data/chen2018-xmmlss-data-products-2018-06-08/xexp_merged.full.v01.fits'
# wcs, hdu = utils.load_wcs(image_file)
# cov_function = vmax.coverage_function(hdu.data, wcs, 3824, 2694, detector_area=36, photon_energy=7e-9)

def cov_func(l, z): 
    flux = l / (4 * np.pi * cosmo.luminosity_distance(z).value**2)
# #     print(flux[0, 0], cov_function(flux[0, 0]))
    return 4.8 * coverage_correction(flux / unit_conversion)

print('here')
vmax_vals = vmax.compute_binned_vmax_values(l, (z, zmin, zmax), l_bins, z_bins, cosmo, bin_z_bounds=False, coverage=cov_func)

In [ ]:
zs = (z_bins[:-1]+z_bins[1:])/2
l_limits = 4 * np.pi * cosmo.luminosity_distance(zs).to(u.cm).value**2 * flux_limit / (1 + zs)**(1 + spectral_index)
print(l_limits)
lf_vals, lf_errs = vmax.compute_lf_values(l, z, vmax_vals, z_bins, l_bins)

In [ ]:
lit_data = {}
hz_convert = 5.8e-6 / 3e8
import importlib
importlib.reload(af)

lacy_ir_evol_params_central = {'A': 10**-4.75, 
                               'gamma1':1.07, 
                               'gamma2':2.48,
                               'Lstar': 10**31.92 / hz_convert,
                               'zref':2.5,
                               'k1':1.05,
                               'k2':-4.71,
                               'k3':-0.034
                              }
lacy_ir_evol_params_max = {'A': 10**-4.73, 
                               'gamma1':1.13, 
                               'gamma2':2.53,
                               'Lstar': 10**31.94 / hz_convert,
                               'zref':2.5,
                               'k1':1.08,
                               'k2':-4.58,
                               'k3':0.156
                              }
lacy_ir_evol_params_min = {'A': 10**-4.77, 
                               'gamma1':1.01, 
                               'gamma2':2.43,
                               'Lstar': 10**31.9 / hz_convert,
                               'zref':2.5,
                               'k1':1.02,
                               'k2':-4.84,
                               'k3':-0.224
                              }

center_zbins = (z_bins[1:] + z_bins[:-1]) / 2
ls = np.logspace(43, 46.2, num=50)
lacy_mid = af.IR_evol(ls, center_zbins, **lacy_ir_evol_params_central)
lacy_high = af.IR_evol(ls, center_zbins, **lacy_ir_evol_params_max)
lacy_low = af.IR_evol(ls, center_zbins, **lacy_ir_evol_params_min)
# print(lacy_mid[0])
lit_data['Lacy 2015'] = list(zip(lacy_mid, lacy_high, lacy_low))
# lacy_mid[1][:10]

In [ ]:
fig, axs, big_ax = vmax.plot_lf_vmax(lf_vals, lf_errs, z_bins, l_bins, \
                                     lum_limits=l_limits, compare_to_others=lit_data, \
                                     title=r'LF for 5.8 $\mu$m', \
                                     outfile='./output/ir_lf.png', lum_sublabel='_{{5.8}}')

In [ ]:
from astropy import cosmology

In [ ]:
greene = cosmology.LambdaCDM(H0=71, Om0=0.27, Ode0=0.75)
farrah = cosmology.LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)
medling = cosmology.LambdaCDM(H0=70, Om0=0.28, Ode0=0.72)

In [ ]:
z=1
farrah.luminosity_distance(z)**2/greene.luminosity_distance(z)**2, farrah.luminosity_distance(z)**2/medling.luminosity_distance(z)**2